In [1]:
import csv
import os
import glob
import pandas as pd
import nltk
import string
import re
from nltk import ngrams
import multiprocessing
import time

In [2]:
def clean_and_split_str(string):
    strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    string = re.sub(strip_special_chars, " ", string)
    string = string.lower()
    return string

In [3]:
def find_ngrams(sentence, n_list):
    """Magic n-gram function."""
    inp, grams = sentence.split(), []
    for n in n_list:
      grams += [' '.join(x) for x in zip(*[inp[i:] for i in range(n)])]
    return grams

In [7]:
os.chdir("C://Users//Ruben//Documents//Scriptie//Data//tng-environment//leyc")
list_csv = glob.glob("*.csv")
list_csv

['leyc-1815-1868.csv']

In [ ]:
# Function for extracting ngrams and generating tokenized files

def PreProcess(filename):
    # Iterate over CSVs
    df = pd.read_csv(filename, sep = "\t")
    print(filename + " imported")
    df.date = df.date.str.slice(0, 4)
    

    # Iterate over years
    for year in sorted(list(set(df.date))):
        df_subset_year = df[df.date == year]
        df_subset_year = df_subset_year.reset_index(drop=True)
        print(str(year) + " = " + str(len(df_subset_year)) + " articles")
        bigram_df = pd.DataFrame()
        unigram_df = pd.DataFrame()
        
        list_tok_articles_year = list()
        
        # Iterate over Articles in Subsetted CSV
        for i in range(0,(len(df_subset_year.ocr) - 1)):
            
            
            article = df_subset_year.ocr[i]
            
            tokens = clean_and_split_str(article)
            list_tok_articles_year.append(tokens)
            
            
            # Get ngrams from tokenized_article
            list_ngrams = find_ngrams(tokens, [1,2])
            list_unigrams = [word for word in list_ngrams if len(word.split(" ")) == 1]
            list_bigrams = [word for word in list_ngrams if len(word.split(" ")) == 2]

        
            df_unigrams_article = pd.DataFrame(list_unigrams)
            df_unigrams_article['year'] = year
            df_unigrams_article['count'] = 1
            df_unigrams_article.columns = ['ngram', 'year', 'count']
            unigram_df = unigram_df.append(df_unigrams_article)
            
            
            df_bigrams_article = pd.DataFrame(list_bigrams)
            df_bigrams_article['year'] = year
            df_bigrams_article['count'] = 1
            df_bigrams_article.columns = ['ngram', 'year', 'count']
            bigram_df = bigram_df.append(df_bigrams_article)
            print(str(i) + "/" + str(len(df_subset_year) + ": " + df_subset_year.date[i] + " " + df_subset_year.id[i] + " processed")
        
        #Write list of senttok articles to one file
        txt_name = filename[0:4] + "_" + "lines" + "_" + str(year) + ".txt"
                        
        with open(txt_name, 'w') as f:
            for item in list_tok_articles_year:
                f.write("%s\n" % item)
        
        ## Group Ngram DFS
        unigram_df = unigram_df.groupby(['ngram', 'year']).sum()
        unigram_df = unigram_df.reset_index()
        unigram_df = unigram_df[unigram_df['count'] > 1]
        fn = filename[0:4] + "_" + "unigram" "_" + str(year) +".csv"
        unigram_df.to_csv(fn, index = False)
    
        bigram_df = bigram_df.groupby(['ngram', 'year']).sum()
        bigram_df = bigram_df.reset_index()
        bigram_df = bigram_df[bigram_df['count'] > 1]
        fn = filename[0:4] + "_" + "bigram" + "_" + str(year) +".csv"
        bigram_df.to_csv(fn, index = False)

In [ ]:
PreProcess(list_csv[0])